In [5]:
import re
from pathlib import Path


def leer(origen: str = "alumnos.md") -> list:
    """
    Lee un archivo markdown con datos de alumnos y devuelve una lista de diccionarios.
    """

    print(f"> Leyendo {origen}...")
    salida, comision = [], ""
    with open(origen, "r", encoding='utf-8') as archivo:
        for linea in archivo:
            if not linea.strip():
                continue

            if linea.startswith("##"):
                comision = "C" + linea.strip("# \n")[-1]

            if linea.startswith("- "):
                valores = re.split(r"\s{2,}", linea.strip("- \n"))
                if len(valores) == 3: valores.append("c")
                valores.append(comision)
                valores = dict(zip('legajo nombre telefono estado comision'.split(), valores))
                salida.append(valores)

    return sorted(salida, key=lambda x: [x['comision'], x['legajo']])


def escribir(alumnos, destino="alumnos2.md"):
    """ 
    Escribe un archivo markdown con datos de alumnos a partir de una lista de diccionarios.
    """

    print(f"> Escribiendo {destino}...")
    with open(destino, "w", encoding='utf-8') as archivo:
        archivo.write("# Programación 4 | TUP 25\n")
        for comision in sorted(set(alumno['comision'] for alumno in alumnos)):
            archivo.write(f"\n## Comisión {comision[-1]}\n\n")
            archivo.write("```text\n")
            for alumno in alumnos:
                if alumno['comision'] == comision:
                    archivo.write(f"- {alumno['legajo']}  {alumno['nombre']:40}  {alumno['telefono']}  {alumno['estado']}\n")
            archivo.write("```\n")
    return alumnos


def convertir_vcard(alumnos, archivo_destino="contactos_alumnos.vcf"):
    """
    Convierte una lista de alumnos en un archivo VCard compatible con WhatsApp.
    """

    print(f"> Generando archivo VCard: {archivo_destino}...")
    
    with open(archivo_destino, "w", encoding='utf-8') as archivo:
        for alumno in alumnos:
            # Separar nombre y apellidos
            nombre_completo = alumno['nombre'].strip()
            partes_nombre = nombre_completo.split(', ')
            
            if len(partes_nombre) == 2:
                # Formato: "Apellido, Nombre"
                apellidos = partes_nombre[0].strip()
                nombres = partes_nombre[1].strip()
            else:
                # Si no tiene el formato esperado, usar todo como nombre
                nombres = nombre_completo
                apellidos = ""
            
            # Limpiar el número de teléfono (remover espacios y caracteres especiales)
            telefono = re.sub(r'[^\d+]', '', alumno['telefono'])
            
            # Generar VCard
            vcard = [
                "BEGIN:VCARD",
                "VERSION:3.0",
                f"FN:{nombre_completo}",  # Nombre completo para mostrar
                f"N:{apellidos};{nombres};;;",  # Apellido;Nombre;SegundoNombre;Prefijo;Sufijo
                f"TEL;TYPE=CELL:{telefono}",  # Teléfono celular
                f"NOTE:Legajo: {alumno['legajo']} - Comision: P4-{alumno['comision']}",  # Nota con legajo y comisión
                "END:VCARD"
            ]
            
            # Escribir el VCard al archivo
            archivo.write("\n".join(vcard) + "\n\n")
    
    print(f"Archivo generado con {len(alumnos)} contactos.")
    return len(alumnos)


def normalizar(origen='alumnos.md', orden='nombre'):
    """ Lee un archivo markdown con datos de alumnos, los ordena y escribe en otro archivo.
        - orden: `nombre` | `legajo`
    """
    alumnos = leer(origen)
    alumnos = sorted(alumnos, key=lambda x: [x['comision'], x[orden]])
    return escribir(alumnos, origen)


def carpeta(alumno, tp=None): 
    """ Calcula el nombre de la carpeta para un alumno. """
    return f"../tp/{alumno['legajo']} - {alumno['nombre']}{'' if tp is None else f'/tp{tp}'}"


def crear_carpetas(alumnos):
    """Crea carpetas para cada alumno (tp/<legajo> - <nombre>) y un archivo info.txt dentro."""
    raiz = Path('tp')
    for a in alumnos:
        destino = Path(carpeta(a))  # ya incluye 'tp/...'
        # Si existe carpeta con nombre viejo (mismo legajo) renombrar
        for viejo in raiz.glob(f"{a['legajo']} -*"):
            if viejo != destino:
                viejo.rename(destino)
        # Asegurar existencia y escribir info.txt
        destino.mkdir(parents=True, exist_ok=True)
        (destino / "info.txt").write_text(
            f"Legajo: {a['legajo']}\nNombre: {a['nombre']}\nTeléfono: {a['telefono']}\nComisión: {a['comision']}\n",
            encoding='utf-8'
        )


normalizar()

# crear_carpetas(leer())

> Leyendo alumnos.md...
> Escribiendo alumnos.md...


[{'legajo': '61131',
  'nombre': 'Agostino Colombres, Juan Manuel',
  'telefono': '(381) 684-8399',
  'estado': 'a',
  'comision': 'C1'},
 {'legajo': '62171',
  'nombre': 'Ballespin, Cristian',
  'telefono': '(381) 604-5734',
  'estado': 'c',
  'comision': 'C1'},
 {'legajo': '99001',
  'nombre': 'Branda, Cesar',
  'telefono': '(381) 212-6932',
  'estado': 'c',
  'comision': 'C1'},
 {'legajo': '61565',
  'nombre': 'Carrizo, Mauro',
  'telefono': '(381) 610-8745',
  'estado': 'a',
  'comision': 'C1'},
 {'legajo': '61551',
  'nombre': 'Diaz Valdez, Ignacio',
  'telefono': '(381) 695-8566',
  'estado': 'a',
  'comision': 'C1'},
 {'legajo': '61472',
  'nombre': 'Diaz, Julieta Camila',
  'telefono': '(381) 652-9981',
  'estado': 'x',
  'comision': 'C1'},
 {'legajo': '61236',
  'nombre': 'Drachenberg, Franco Eduardo',
  'telefono': '(381) 670-8708',
  'estado': 'a',
  'comision': 'C1'},
 {'legajo': '61337',
  'nombre': 'Díaz, Christian Gabriel',
  'telefono': '(381) 571-4797',
  'estado': 'a'

In [6]:
import shutil
from pathlib import Path

def copiar_tp(tp=1, alumnos=None):
    """ Copia todos los archivos y subcarpetas de enunciados/tp{tp}
        a la carpeta correspondiente de cada alumno (tp/<legajo> - <nombre>/tp{tp}).
    """
    if alumnos is None:
        alumnos = leer()

    origen_base = Path(f"../enunciados/tp{tp}")
    if not origen_base.exists():
        print(f"No existe {origen_base}")
        return

    for a in alumnos:
        destino_base = Path(carpeta(alumno=a, tp=tp))  # ya incluye 'tp/.../tp{tp}'
        destino_base.mkdir(parents=True, exist_ok=True)

        print(f"\n==> Copiando en {destino_base}")

        for origen in origen_base.rglob('*'):
            rel = origen.relative_to(origen_base)
            destino = destino_base / rel

            destino.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(origen, destino)
            print(f"Copiado: {origen} -> {destino}")


alumnos = leer()
crear_carpetas(alumnos)
copiar_tp(1, alumnos)

> Leyendo alumnos.md...

==> Copiando en ../tp/54911 - Jatib, Rodrigo Gabriel/tp1
Copiado: ../enunciados/tp1/io.js -> ../tp/54911 - Jatib, Rodrigo Gabriel/tp1/io.js
Copiado: ../enunciados/tp1/enunciado.md -> ../tp/54911 - Jatib, Rodrigo Gabriel/tp1/enunciado.md
Copiado: ../enunciados/tp1/agenda.json -> ../tp/54911 - Jatib, Rodrigo Gabriel/tp1/agenda.json
Copiado: ../enunciados/tp1/ejercicio.js -> ../tp/54911 - Jatib, Rodrigo Gabriel/tp1/ejercicio.js

==> Copiando en ../tp/61033 - Quiroga, Jose Maria/tp1
Copiado: ../enunciados/tp1/io.js -> ../tp/61033 - Quiroga, Jose Maria/tp1/io.js
Copiado: ../enunciados/tp1/enunciado.md -> ../tp/61033 - Quiroga, Jose Maria/tp1/enunciado.md
Copiado: ../enunciados/tp1/agenda.json -> ../tp/61033 - Quiroga, Jose Maria/tp1/agenda.json
Copiado: ../enunciados/tp1/ejercicio.js -> ../tp/61033 - Quiroga, Jose Maria/tp1/ejercicio.js

==> Copiando en ../tp/61035 - Ledesma, Paulo Marcelo/tp1
Copiado: ../enunciados/tp1/io.js -> ../tp/61035 - Ledesma, Paulo Marcelo/

In [7]:
# Ejemplo de uso de la función convertir_vcard
alumnos = leer('alumnos.md')
print(f"Se encontraron {len(alumnos)} alumnos")

# Generar archivo VCard con todos los alumnos
contactos_generados = convertir_vcard(alumnos, "alumnos.vcf")

print("Archivos VCard generados exitosamente!")
print("Puedes importar estos archivos directamente en WhatsApp o en tu aplicación de contactos.")

> Leyendo alumnos.md...
Se encontraron 82 alumnos
> Generando archivo VCard: alumnos.vcf...
Archivo generado con 82 contactos.
Archivos VCard generados exitosamente!
Puedes importar estos archivos directamente en WhatsApp o en tu aplicación de contactos.


In [8]:
# import pandas as pd
# p = "./alumnos.csv"


# # Leer como cadenas (evita problemas con tipos mixtos), manejar comillas internas y espacios
# c = pd.read_csv(p, dtype=str, encoding="utf-8", skipinitialspace=True, keep_default_na=False)
# a = pd.DataFrame(leer())

# c['legajo'] = c['legajo'].astype(str).str.strip()
# a['legajo'] = a['legajo'].astype(str).str.strip()

# faltantes = c[~c['legajo'].isin(a['legajo'])]
# faltantes.to_csv("alumnos_faltantes(c-a).csv", index=False, encoding="utf-8")
# faltantes = a[~a['legajo'].isin(c['legajo'])]
# faltantes.to_csv("alumnos_faltantes(a-c).csv", index=False, encoding="utf-8")
# a.to_csv("alumnos_a.csv", index=False, encoding="utf-8")
# c.to_csv("alumnos_c.csv", index=False, encoding="utf-8")